In [58]:
# Setup Julia environment
using Pkg
Pkg.activate(".")
using ITensors
using LinearAlgebra
using Printf
using Statistics

# Load BOTH BinderSim modules
include("../src/BinderSim.jl")
include("../src/BinderSimForced.jl")


  Activating new project at `~/Desktop/binder-parameter-OSpool/src`


Main.BinderSimForced

## Setup and Load Modules

In [59]:
L_test = 3
lambda_x_test = 0.3
lambda_zz_test = 0.7

println("Validation Parameters:")
println("L = $L_test")
println("λₓ = $lambda_x_test") 
println("λ_zz = $lambda_zz_test")
println("T_max = $(2*L_test)")

hand_binder_expected = 0.65980113

Validation Parameters:
L = 3
λₓ = 0.3
λ_zz = 0.7
T_max = 6


0.65980113

In [63]:
using LinearAlgebra

function create_pauli_matrices()
    I = [1 0; 
        0 1]

    X = [0 1; 
        1 0] 

    Z = [1 0; 
        0 -1]
    return I, X, Z
end

function construct_weak_measurement_operators(lambda_x, lambda_zz)
    I, X, Z = create_pauli_matrices()
    
    norm_x = sqrt(2 * (1 + lambda_x^2))
    norm_zz = sqrt(2 * (1 + lambda_zz^2))
    
    K_X_1 = (I - lambda_x * X) / norm_x
    
    II = kron(I, I)
    ZZ = kron(Z, Z)
    K_ZZ_1 = (II - lambda_zz * ZZ) / norm_zz
    
    return K_X_1, K_ZZ_1, norm_x, norm_zz
end

function create_initial_state_3spin()
    # |↑↑↑⟩ = |000⟩ in computational basis
    up = [1; 0]
    return kron(kron(up, up), up)  # 8-component vector
end

function apply_single_site_measurement(psi, K_op, site_idx)
    I = [1 0; 
        0 1]
    
    if site_idx == 0
        full_op = kron(kron(K_op, I), I)
    elseif site_idx == 1  
        full_op = kron(kron(I, K_op), I)
    elseif site_idx == 2
        full_op = kron(kron(I, I), K_op)
    end
    
    psi_new = full_op * psi
    return psi_new / norm(psi_new)
end

function apply_two_site_measurement(psi, K_op, bond)
    # Apply K_op to bond (0,1) or (1,2)  
    I = [1 0; 
        0 1]
    
    if bond == (0,1)
        full_op = kron(K_op, I)
    elseif bond == (1,2)
        full_op = kron(I, K_op)
    end
    
    psi_new = full_op * psi
    return psi_new / norm(psi_new)
end

function evolve_3spin_hand_calculation(lambda_x, lambda_zz, T_max=6)
    # Initial state
    psi = create_initial_state_3spin()
    
    # Get operators
    K_X_1, K_ZZ_1, norm_x, norm_zz = construct_weak_measurement_operators(lambda_x, lambda_zz)
    
    println("Hand calculation evolution:")
    println("Initial state norm: $(norm(psi))")
    
    for t in 1:T_max
        # X measurements on all sites (forced +1)
        for i in 0:2
            psi = apply_single_site_measurement(psi, K_X_1, i)
        end
        
        # Simultaneous ZZ measurements on bonds (0,1) and (1,2)
        psi = apply_two_site_measurement(psi, K_ZZ_1, (0,1))
        psi = apply_two_site_measurement(psi, K_ZZ_1, (1,2))
        
        println("After step $t: norm = $(norm(psi))")
    end
    
    return psi
end

function calculate_correlators_3spin(psi)
    I = [1 0; 
        0 1]
    Z = [1 0; 
        0 -1]
    
    # Single-site Z operators
    Z0 = kron(kron(Z, I), I)
    Z1 = kron(kron(I, Z), I) 
    Z2 = kron(kron(I, I), Z)
    
    # Calculate all 2-point correlators ⟨Z_i Z_j⟩
    correlators_2pt = Dict()
    for i in 0:2, j in 0:2
        Zi = [Z0, Z1, Z2][i+1]
        Zj = [Z0, Z1, Z2][j+1]
        correlators_2pt[(i,j)] = real(psi' * (Zi * Zj) * psi)
    end
    
    # Calculate all 4-point correlators ⟨Z_i Z_j Z_k Z_l⟩
    correlators_4pt = Dict()
    for i in 0:2, j in 0:2, k in 0:2, l in 0:2
        Zi = [Z0, Z1, Z2][i+1]
        Zj = [Z0, Z1, Z2][j+1]
        Zk = [Z0, Z1, Z2][k+1]
        Zl = [Z0, Z1, Z2][l+1]
        correlators_4pt[(i,j,k,l)] = real(psi' * (Zi * Zj * Zk * Zl) * psi)
    end
    
    return correlators_2pt, correlators_4pt
end

function calculate_binder_parameter_hand(correlators_2pt, correlators_4pt, L=3)
    # M2sq = ∑ᵢⱼ |⟨Z_i Z_j⟩|² / L²
    M2sq = sum(abs(v)^2 for v in values(correlators_2pt)) / L^2
    
    # M4sq = ∑ᵢⱼₖₗ |⟨Z_i Z_j Z_k Z_l⟩|² / L⁴  
    M4sq = sum(abs(v)^2 for v in values(correlators_4pt)) / L^4
    
    # Binder parameter: B = 1 - M4sq/(3*M2sq²)
    B = 1 - M4sq / (3 * M2sq^2)
    
    return B, M2sq, M4sq
end

psi_final = evolve_3spin_hand_calculation(lambda_x_test, lambda_zz_test, 6)
corr_2pt, corr_4pt = calculate_correlators_3spin(psi_final)
B_hand_calculated, M2sq_hand_calculated, M4sq_hand_calculated = calculate_binder_parameter_hand(corr_2pt, corr_4pt, 3)

println("M2sq = $(M2sq_hand_calculated)")
println("M4sq = $(M4sq_hand_calculated)")  
println("B = $(B_hand_calculated)")


hand_binder_expected = B_hand_calculated

Hand calculation evolution:
Initial state norm: 1.0
After step 1: norm = 0.9999999999999999
After step 2: norm = 1.0
After step 3: norm = 1.0
After step 4: norm = 1.0
After step 5: norm = 1.0
After step 6: norm = 1.0
M2sq = 0.9772890680045975
M4sq = 0.9747656311162186
B = 0.6598011259372127
Initial state norm: 1.0
After step 1: norm = 0.9999999999999999
After step 2: norm = 1.0
After step 3: norm = 1.0
After step 4: norm = 1.0
After step 5: norm = 1.0
After step 6: norm = 1.0
M2sq = 0.9772890680045975
M4sq = 0.9747656311162186
B = 0.6598011259372127


0.6598011259372127

## Test BinderSimForced (Deterministic)

In [65]:
result_forced = BinderSimForced.ea_binder_mc_forced(L_test; lambda_x=lambda_x_test, lambda_zz=lambda_zz_test, 
                                                    ntrials=1, seed=12345)


# Store for comparison
julia_forced_binder = result_forced.B

Edwards-Anderson Binder parameter with FORCED +1 measurements
L = 3, λₓ = 0.3, λ_zz = 0.7
Note: All measurements forced to +1 outcome (deterministic)
\nTrial 1/1:
  Evolving with forced +1 measurements for 6 time steps...
    Time step 1/6, max bond dimension: 2
    Time step 2/6, max bond dimension: 2
    Time step 3/6, max bond dimension: 2
    Time step 4/6, max bond dimension: 2
    Time step 5/6, max bond dimension: 2
    Time step 6/6, max bond dimension: 2
  Computing correlators on ALL sites: 1 to 3 (3 sites)
  Computing 9 2-point and 81 4-point correlators...
  Trial 1 results: M2sq = 0.9772890680045949, M4sq = 0.9747656311162174, B = 0.6598011259372114
\nFinal ensemble results:
  Ensemble B_EA = 0.6598011259372114
  Mean of trials = 0.6598011259372114
  Std of trials = NaN


0.6598011259372114

## Test BinderSim (Random Sampling)

In [68]:
result_random = BinderSim.ea_binder_mc(L_test; lambda_x=lambda_x_test, lambda_zz=lambda_zz_test, 
                                       ntrials=50, seed=12345)

println("Binder parameter: ", result_random.B)
println("S2_bar (M2sq): ", result_random.S2_bar)
println("S4_bar (M4sq): ", result_random.S4_bar)
println("B_std_of_trials: ", result_random.B_std_of_trials)

julia_random_binder = result_random.B

Binder parameter: 0.6431439611354937
S2_bar (M2sq): 0.9257604927501145
S4_bar (M4sq): 0.9175116586112382
B_std_of_trials: 0.05053766939450079

S2_bar (M2sq): 0.9257604927501145
S4_bar (M4sq): 0.9175116586112382
B_std_of_trials: 0.05053766939450079


0.6431439611354937